In [3]:
import ibis
from ibis import _

# NEXMark schemas https://datalab.cs.pdx.edu/niagara/NEXMark/
person_schema = ibis.schema(
    {
        "id": "int64",
        "name": "string",
        "emailaddress": "string",
        "creditcard": "string",
        "city": "string",
        "state": "string"
    }
)

auction_schema = ibis.schema(
    {
        "id": "int64",
        "itemname": "string",
        "description": "string",
        "initialbid": "float64",
        "reserve": "float64",
        "expires": "timestamp(3)",
        "seller": "int64",
    }
)

bid_schema = ibis.schema(
    {
        "auction": "int64",
        "bidder": "int64",
        "price": "float64",
        "datetime": "timestamp(3)"
    }
)

# Create SubstraitCompiler object
from ibis_substrait.compiler.core import SubstraitCompiler
compiler = SubstraitCompiler()

In [4]:
# CREATE DDL
# TODO: add support for WITH statements; 
# https://github.com/ibis-project/ibis-substrait/issues/1121

bid_table = ibis.table(name="Bid", schema=bid_schema)
ddl_plan = compiler.compile(bid_table)
ddl_plan

relations {
  root {
    input {
      read {
        common {
          direct {
          }
        }
        base_schema {
          names: "auction"
          names: "bidder"
          names: "price"
          names: "datetime"
          struct {
            types {
              i64 {
                nullability: NULLABILITY_NULLABLE
              }
            }
            types {
              i64 {
                nullability: NULLABILITY_NULLABLE
              }
            }
            types {
              fp64 {
                nullability: NULLABILITY_NULLABLE
              }
            }
            types {
              timestamp {
                nullability: NULLABILITY_NULLABLE
              }
            }
            nullability: NULLABILITY_REQUIRED
          }
        }
        named_table {
          names: "Bid"
        }
      }
    }
    names: "auction"
    names: "bidder"
    names: "price"
    names: "datetime"
  }
}
version {
  minor_number: 54
  produc

In [5]:
# windowed transformation, note this is the standard over window (not the streaming specific )
# For now, we'll pass batch logical plan, see https://github.com/ibis-project/ibis-substrait/issues/1117
w = ibis.window(group_by=_.auction, preceding=5, following=5, order_by=_.datetime)
window_avg_price = bid_table.mutate(avg_price=_.price.mean().over(w))
ibis.to_sql(window_avg_price)


```sql
SELECT
  "t0"."auction",
  "t0"."bidder",
  "t0"."price",
  "t0"."datetime",
  AVG("t0"."price") OVER (PARTITION BY "t0"."auction" ORDER BY "t0"."datetime" ASC ROWS BETWEEN 5 preceding AND 5 following) AS "avg_price"
FROM "Bid" AS "t0"
```

In [6]:
plan = compiler.compile(window_avg_price)
with open("window_avg_price.proto", "wb") as f:
    f.write(plan.SerializeToString())

plan

extension_uris {
  extension_uri_anchor: 1
  uri: "https://github.com/substrait-io/substrait/blob/main/extensions/functions_arithmetic.yaml"
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 1
    name: "avg:fp64"
  }
}
relations {
  root {
    input {
      project {
        common {
          emit {
            output_mapping: 4
            output_mapping: 5
            output_mapping: 6
            output_mapping: 7
            output_mapping: 8
          }
        }
        input {
          read {
            common {
              direct {
              }
            }
            base_schema {
              names: "auction"
              names: "bidder"
              names: "price"
              names: "datetime"
              struct {
                types {
                  i64 {
                    nullability: NULLABILITY_NULLABLE
                  }
                }
                types {
                  i64 {
                   